In [2]:
#webscraping here

import pandas as pd
import praw

reddit = praw.Reddit(client_id="69xO_MRhRBIQGprtVEZIbA",
    client_secret="GgPpkwrp_BPNduxYQ-dOcxfmrykfhQ",
    user_agent="test")


subreddit = reddit.subreddit("scu")

posts_dict = {"Body": [], "ID": [], "Ups": []}

for post in subreddit.hot(limit=1000):
    # Title and body of each post
    title = post.title
    text = post.selftext
    body = title + "\n" + text
    posts_dict['Body'].append(body)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Ups"].append(post.score)

top_posts = pd.DataFrame(posts_dict)
top_posts.to_csv("Top Posts.csv", index=True)
display(top_posts)



Version 7.7.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


,Body,ID,Ups
0,Recruitments for Studies/Job Opportunities & O...,oouus2,18
1,Advice Paper for Incoming COEN/CS Students\nHi...,xkfy3u,46
2,Robotics and Automation in SCU\nI am from Asia...,1dbiqsj,1
3,"CRE QUESTION\nHello,\n\nI’ve took AP Calculus ...",1dbe0b2,2
4,Entire Townhome Available for Sublet//1st Quar...,1db2tb3,1
...,...,...,...
969,Freshman dorms\nHow hard is it to get into Fin...,kxpxqv,5
970,(Open to all Bay Area Residents) 1 Min Survey ...,kwq8y5,7
971,Hackathon Opportunity For SCU Students (All Ma...,ku4vj1,11
972,"Calling all engineering, computer science, and...",ks6uiv,4


In [27]:

id_list = top_posts["ID"].tolist()
#id_list = ["1d5yi68", "1cogl0"]
comments = pd.DataFrame()
comment_list = []
for i in id_list:                      #indicates start of the subission id processing
    submission = reddit.submission(i)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if(comment.author == "AutoModerator"):
            continue
        else:
            comment_list.append(comment.__dict__)

comments_df = pd.DataFrame(comment_list)

cleaned_df = comments_df.filter(['_submission','id','author', 'body', 'ups'], axis=1)

#cleaned_df.to_csv(f'comments.csv')

display(cleaned_df)

TooManyRequests: received 429 HTTP response

In [29]:
import nltk
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

dorm_names = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin-walsh', 'casa italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university villas', 'nobili']

# Function to extract dorm names from comments
def dorm_appears(comment, dorm):
    comment_lower = comment.lower()
    tokens = tokenizer.tokenize(comment_lower)
    return dorm in tokens

# Check if at least one dorm is found in the comment
def any_dorm_found(row):
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_") 
        if row[dorm_column]:
            return True
    return False

for dorm in dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
    #display(cleaned_df['body'])
    top_posts[dorm_column] = top_posts['Body'].apply(dorm_appears, dorm=dorm)

# Remove rows where no dorms  found
top_posts = top_posts[top_posts.apply(any_dorm_found, axis=1)]

display(top_posts)

,Body,ID,Ups,graham,finn,sanfilippo,campisi,mclaughlin_walsh,casa_italiana,sobrato,swig,flip,camp,casa,walsh,university_villas,nobili
38,What are my chances?\nHey everyone!\n\nI'm cur...,1cp9tqh,5,False,False,False,False,False,False,False,False,False,True,False,False,False,False
49,Graham/Finn\nDo incoming freshman have a chanc...,1cmhmzn,3,True,True,False,False,False,False,False,False,False,False,False,False,False,False
67,"Transfer Housing Information\nHello, I'm a tra...",1chxhvo,3,False,True,False,False,False,False,False,False,False,False,False,False,False,False
76,Orientation Sessions are open for registration...,1cd3a17,7,False,False,False,False,False,False,False,True,False,False,False,False,False,False
80,Dunne bathroom updated?\nI am about to submit ...,1camxec,1,False,False,False,False,False,False,False,False,False,False,False,True,False,False
101,"Santa Clara University or UC Santa Cruz, Which...",1c22i9j,2,False,False,False,False,False,False,False,False,True,False,False,False,False,False
114,Complaining about Swig & Swig Residents until ...,1byptue,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False
156,Housing at SCU\nI was wondering how housing wo...,1bmxuwd,2,True,False,False,False,False,False,False,False,False,False,True,False,False,False
183,If you’re a first year honors student are you ...,1bh9dfi,2,False,False,False,False,False,False,False,True,False,False,False,False,False,False
186,"Rank the dorms\nPlease rank\n\nCasa, campesi, ...",1bf62mi,7,False,False,True,False,False,False,True,False,False,False,True,False,False,False
